In [36]:
import rouge

rouge = rouge.Rouge()
# Assume that the correct modification is black -> red
hyp_list = [
    'A black fox', # identical
    'A  black\t fox', # identical but some format differences
    'A red fox', # correct update
    'A cute black fox jumps over the lazy dog', # extend only
    'A cute red fox jumps over the lazy dog', # correct update while extend more
    'A crimson vulpine', # Correct but close to generation instead of updating
]
ref = ['A black fox'] * len(hyp_list)

scores = rouge.get_scores(refs=ref, hyps=hyp_list)
score = scores[0]['rouge-l']['r'] # focus on the recall point

print(f"Original: {ref[0]}")
print(f"Expected: {hyp_list[2]}")
for i, score in enumerate(scores):
    hyp = hyp_list[i]
    recall = score['rouge-l']['r']
    precision = score['rouge-l']['p']
    f = score['rouge-l']['f']
    print(f'p: {precision:.2f} r: {recall:.2f} f: {f:.2f} - {hyp}')

Original: A black fox
Expected: A red fox
p: 1.00 r: 1.00 f: 1.00 - A black fox
p: 1.00 r: 1.00 f: 1.00 - A  black	 fox
p: 0.67 r: 0.67 f: 0.67 - A red fox
p: 0.33 r: 1.00 f: 0.50 - A cute black fox jumps over the lazy dog
p: 0.22 r: 0.67 f: 0.33 - A cute red fox jumps over the lazy dog
p: 0.33 r: 0.33 f: 0.33 - A crimson vulpine
